# OpenCV Docs
Camera Calibration and 3D reconstruction:

https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

Detection of ArUco Markers:

https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html

# Configuration

In [1]:
# Dependencies
import cv2
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

# Configuration
FRAME_WIDTH = 1280 # [px]
FRAME_HEIGHT = 720 # [px]
ARUCO_DICT = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
NUM_MARKERS = 50
MARKER_SIZE = 140 # [px]
MARKER_BORDER_WIDTH = 20 # [%]
MARKER_FOLDER = "markers"
CHESSBOARD_DIMENSIONS = (10, 7)
CHESSBOARD_SQUARE_SIZE = 80 # [px]
CHESSBOARD_FILENAME = "chessboard.png"

# Generate ArUco Markers

In [7]:
# Create the output folder if it doesn't already exist
if not os.path.exists(MARKER_FOLDER):
    os.makedirs(MARKER_FOLDER)
    print(f"Folder '{MARKER_FOLDER}' created.")

print(f"Generating {NUM_MARKERS} markers...")

# Loop to generate and save markers (IDs 0 to NUM_MARKERS-1)
for marker_id in range(NUM_MARKERS):
    # 1. Generate the base marker image (the black and white square)
    marker_image = cv2.aruco.generateImageMarker(ARUCO_DICT, marker_id, MARKER_SIZE)

    # 2. Calculate the border size in pixels
    border_pixels = int(MARKER_SIZE * MARKER_BORDER_WIDTH / 100)

    # 3. Create a new, larger image (the canvas) and fill it with white
    # The new total size is the original size plus the border on both sides (left/right or top/bottom)
    new_size = MARKER_SIZE + 2 * border_pixels
    bordered_image = np.full((new_size, new_size), 255, dtype=np.uint8)

    # 4. Copy the generated marker onto the center of the white canvas
    # We calculate the region where the marker should be placed
    start_point = border_pixels
    end_point = border_pixels + MARKER_SIZE
    bordered_image[start_point:end_point, start_point:end_point] = marker_image

    # 5. Define the filename and save the new image
    file_name = os.path.join(MARKER_FOLDER, f"marker-{marker_id}.png")
    cv2.imwrite(file_name, bordered_image)

print("-" * 30)
print(f"Generated {NUM_MARKERS} markers, {MARKER_SIZE}x{MARKER_SIZE} pixels with a {MARKER_BORDER_WIDTH}% white border.")
print(f"Successfully saved markers to '{os.path.abspath(MARKER_FOLDER)}'")

Generating 50 markers...
------------------------------
Generated 50 markers, 140x140 pixels with a 20% white border.
Successfully saved markers to 'C:\Users\kwiat\Documents\GitHub\Unitree-G1-3D-position-tracking\markers'


# Generate Chessboard

In [8]:
print(f"Generating a {CHESSBOARD_DIMENSIONS[0]}x{CHESSBOARD_DIMENSIONS[1]} chessboard pattern...")

# Calculate the total size of the image in pixels
img_width = CHESSBOARD_DIMENSIONS[0] * CHESSBOARD_SQUARE_SIZE
img_height = CHESSBOARD_DIMENSIONS[1] * CHESSBOARD_SQUARE_SIZE

# Create a new blank image (3 channels for BGR color) and fill it with white
# White in BGR is (255, 255, 255)
chessboard = np.full((img_height, img_width, 3), 255, dtype=np.uint8)

# Loop through each square of the board to draw the pattern
for row in range(CHESSBOARD_DIMENSIONS[1]):
    for col in range(CHESSBOARD_DIMENSIONS[0]):
        # Determine if the square should be black or white
        # If the sum of the row and column is an even number, color the square black
        if (row + col) % 2 == 0:
            # Calculate the top-left corner of the square
            top_left_x = col * CHESSBOARD_SQUARE_SIZE
            top_left_y = row * CHESSBOARD_SQUARE_SIZE
            
            # Calculate the bottom-right corner of the square
            bottom_right_x = top_left_x + CHESSBOARD_SQUARE_SIZE
            bottom_right_y = top_left_y + CHESSBOARD_SQUARE_SIZE
            
            # Draw a filled black rectangle on the image
            # Color for black is (0, 0, 0)
            cv2.rectangle(
                chessboard, 
                (top_left_x, top_left_y), 
                (bottom_right_x, bottom_right_y), 
                (0, 0, 0), 
                -1  # A thickness of -1 fills the rectangle
            )

# Save the final generated image to a file
print("-" * 30)
try:
    cv2.imwrite(CHESSBOARD_FILENAME, chessboard)
    print(f"Successfully saved chessboard pattern to '{os.path.abspath(CHESSBOARD_FILENAME)}'")
except Exception as e:
    print(f"Error: Could not save the image. {e}")

# Optional: Display the image in a window until a key is pressed
cv2.imshow('Generated Chessboard', chessboard)
cv2.waitKey(0)
cv2.destroyAllWindows()


Generating a 10x7 chessboard pattern...
------------------------------
Successfully saved chessboard pattern to 'C:\Users\kwiat\Documents\GitHub\Unitree-G1-3D-position-tracking\chessboard.png'


# Helper Functions

In [2]:
def select_camera():
    """
    Detects available cameras and prompts the user to select one.
    Returns the selected camera index or None if no cameras are found.
    """
    print("Detecting available cameras...")
    available_cameras = []
    for i in range(5):
        cap = cv2.VideoCapture(i, cv2.CAP_DSHOW)
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()

    if not available_cameras:
        print("Error: No cameras found.")
        return None
    
    if len(available_cameras) == 1:
        print(f"Only one camera found (index {available_cameras[0]}). Selecting it automatically.")
        return available_cameras[0]

    print("Multiple cameras found. Please select one:")
    for index in available_cameras:
        print(f"- Enter '{index}' for camera {index}")
    
    while True:
        try:
            choice = int(input("Your choice: "))
            if choice in available_cameras:
                return choice
            else:
                print("Invalid choice. Please select from the available indices.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# Camera Distortion Correction Callibration

In [10]:
# --- Configuration ---
# You need a chessboard pattern. The numbers here are the number of *internal* corners.
# For a standard 9x6 board, you'd use (8, 5). For a 10x7 board, use (9, 6).
chessboard_corners = (CHESSBOARD_DIMENSIONS[0] - 1, CHESSBOARD_DIMENSIONS[1] - 1)
# The name of the file where calibration data will be saved.
CALIBRATION_FILE = "calibration_data.json"
# The folder where captured images will be saved.
CALIBRATION_IMAGES_FOLDER = "calibration-images"

# --- Main Calibration Logic ---
print("Starting Camera Calibration...")
print(f"--> Looking for a {chessboard_corners[0]}x{chessboard_corners[1]} chessboard pattern.")
print("--> Press the [SPACE] bar to capture an image.")
print("--> You need at least 15 good images from different angles and distances.")
print("--> Press [c] to calibrate once you have enough images.")
print("--> Press [q] to quit.")

# Create the output folder for images if it doesn't exist
if not os.path.exists(CALIBRATION_IMAGES_FOLDER):
    os.makedirs(CALIBRATION_IMAGES_FOLDER)
    print(f"Folder '{CALIBRATION_IMAGES_FOLDER}' created.")

# Termination criteria for the corner sub-pixel algorithm
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((chessboard_corners[0] * chessboard_corners[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboard_corners[0], 0:chessboard_corners[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# (Assuming the select_camera() function is defined elsewhere in your notebook)
camera_index = select_camera() 
if camera_index is None:
    raise SystemExit("No camera selected. Exiting.")

cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)

# --- Set the camera resolution ---
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

images_captured = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Display the current frame
    display_frame = frame.copy()
    cv2.putText(display_frame, f"Images Captured: {images_captured}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Calibration', display_frame)

    key = cv2.waitKey(1) & 0xFF
    
    if key == ord(' '): # Space bar to capture
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, chessboard_corners, None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            # Save the captured frame BEFORE drawing on it
            image_filename = os.path.join(CALIBRATION_IMAGES_FOLDER, f"calibration-image-{images_captured}.png")
            cv2.imwrite(image_filename, frame)

            images_captured += 1
            print(f"Image {images_captured} captured and saved to {image_filename}.")
            
            objpoints.append(objp)
            
            # Refine corner locations for better accuracy
            corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)
            
            # Draw and display the corners on the frame for visualization
            cv2.drawChessboardCorners(frame, chessboard_corners, corners2, ret)
            cv2.imshow('Last Capture', frame)
        else:
            print("Could not find chessboard. Try a different angle.")

    elif key == ord('c') and images_captured >= 15:
        print("\nCalibrating camera... this may take a moment.")
        
        # Perform camera calibration
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

        if ret:
            print("Calibration successful!")
            # Create a dictionary to hold the calibration data
            # Convert numpy arrays to lists to make them JSON serializable
            calibration_data = {
                'camera_matrix': mtx.tolist(),
                'distortion_coefficients': dist.tolist()
            }
            
            # Save the data to a JSON file
            with open(CALIBRATION_FILE, 'w') as f:
                json.dump(calibration_data, f, indent=4)
                
            print(f"Calibration data saved to '{CALIBRATION_FILE}'")
            break
        else:
            print("Calibration failed. Please try again with more diverse images.")

        if key == ord('q'):
            break
    
        try:
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        except cv2.error:
            break

cap.release()
cv2.destroyAllWindows()


Starting Camera Calibration...
--> Looking for a 9x6 chessboard pattern.
--> Press the [SPACE] bar to capture an image.
--> You need at least 15 good images from different angles and distances.
--> Press [c] to calibrate once you have enough images.
--> Press [q] to quit.
Detecting available cameras...
Multiple cameras found. Please select one:
- Enter '0' for camera 0
- Enter '1' for camera 1


Your choice:  0


Image 1 captured and saved to calibration-images\calibration-image-0.png.
Image 2 captured and saved to calibration-images\calibration-image-1.png.
Image 3 captured and saved to calibration-images\calibration-image-2.png.
Image 4 captured and saved to calibration-images\calibration-image-3.png.
Image 5 captured and saved to calibration-images\calibration-image-4.png.
Image 6 captured and saved to calibration-images\calibration-image-5.png.
Image 7 captured and saved to calibration-images\calibration-image-6.png.
Image 8 captured and saved to calibration-images\calibration-image-7.png.
Image 9 captured and saved to calibration-images\calibration-image-8.png.
Image 10 captured and saved to calibration-images\calibration-image-9.png.
Image 11 captured and saved to calibration-images\calibration-image-10.png.
Image 12 captured and saved to calibration-images\calibration-image-11.png.
Image 13 captured and saved to calibration-images\calibration-image-12.png.
Image 14 captured and saved to 

# Live Webcam Feed

In [11]:
import cv2
import numpy as np
import os
import json

# --- Configuration ---
NUM_MARKERS = 50
FRAME_WIDTH = 1920
FRAME_HEIGHT = 1080
# The file containing the camera calibration data
CALIBRATION_FILE = "calibration_data.json"
ARUCO_DICT = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

# --- Main Program ---

# Step 1: Load Camera Calibration Data from JSON
if not os.path.exists(CALIBRATION_FILE):
    print(f"Error: Calibration file '{CALIBRATION_FILE}' not found.")
    print("Please run the 'Camera Calibration Script' first.")
    mtx, dist = None, None
else:
    with open(CALIBRATION_FILE, 'r') as f:
        calibration_data = json.load(f)
        # Convert lists from JSON back to numpy arrays
        mtx = np.array(calibration_data['camera_matrix'])
        dist = np.array(calibration_data['distortion_coefficients'])
    print("Camera calibration data loaded successfully.")

# Proceed only if calibration data was loaded
if mtx is not None and dist is not None:
    positions_history = []

    # (Assuming the select_camera() function is defined elsewhere in your notebook)
    camera_index = select_camera()
    if camera_index is None:
        raise SystemExit("No camera selected. Exiting.")

    cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
    
    if not cap.isOpened():
        raise IOError(f"Cannot open webcam at index {camera_index}")

    parameters = cv2.aruco.DetectorParameters()
    parameters.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    detector = cv2.aruco.ArucoDetector(ARUCO_DICT, parameters)
    
    window_name = 'Live Undistorted Feed with Marker Logging'
    h, w = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Step 2: Apply the distortion correction to the frame
        undistorted_frame = cv2.undistort(frame, mtx, dist, None, newcameramtx)
        
        # Crop the image to the valid region (removes black borders)
        x, y, w_roi, h_roi = roi
        undistorted_frame = undistorted_frame[y:y+h_roi, x:x+w_roi]

        # --- Detection now runs on the corrected frame ---
        frame_positions = [None] * NUM_MARKERS
        gray = cv2.cvtColor(undistorted_frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = detector.detectMarkers(gray)

        if ids is not None:
            for i, marker_id in enumerate(ids.flatten()):
                if 0 <= marker_id < NUM_MARKERS:
                    marker_corners = corners[i].reshape((4, 2))
                    cx = int(np.mean(marker_corners[:, 0]))
                    cy = int(np.mean(marker_corners[:, 1]))
                    frame_positions[marker_id] = (cx, cy)
                    
                    cv2.polylines(undistorted_frame, [marker_corners.astype(np.int32)], True, (0, 255, 0), 2)
                    cv2.circle(undistorted_frame, (cx, cy), 5, (0, 0, 255), -1)
                    cv2.putText(undistorted_frame, str(marker_id), (int(marker_corners[0, 0]), int(marker_corners[0, 1]) - 15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        positions_history.append(frame_positions)
        cv2.imshow(window_name, undistorted_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        
        try:
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        except cv2.error:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Stream terminated. 'positions_history' is ready for animation.")


Camera calibration data loaded successfully.
Detecting available cameras...
Multiple cameras found. Please select one:
- Enter '0' for camera 0
- Enter '1' for camera 1


Your choice:  0


Stream terminated. 'positions_history' is ready for animation.


# Position tracking 2D visualisation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np


# Configure matplotlib for Jupyter notebook rendering
plt.rcParams["animation.html"] = "jshtml"
# A higher DPI gives a clearer animation. If you hit the size limit again, you can lower this value.
plt.rcParams['figure.dpi'] = 100

# Check if the position data exists and is not empty
if 'positions_history' not in locals() or not positions_history:
    print("Error: The 'positions_history' data was not found or is empty.")
    print("Please run the previous script cell to capture marker data first.")
else:
    # --- Animation Setup ---
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, FRAME_WIDTH)
    ax.set_ylim(0, FRAME_HEIGHT)
    
    # Invert the Y axis to match image coordinates (0,0 at top-left)
    ax.invert_yaxis()
    ax.set_aspect('equal', adjustable='box')
    fig.tight_layout()

    # --- EFFICIENT ARTIST INITIALIZATION ---
    # Initialize the scatter plot for the marker centers
    scatter = ax.scatter([], [], s=50, c='red', zorder=3)
    
    # Create a pool of text artists, one for each marker.
    # They are initially invisible and will be updated each frame.
    texts = []
    for i in range(NUM_MARKERS):
        text = ax.text(0, 0, str(i), color='blue', fontsize=9, visible=False, zorder=4)
        texts.append(text)
        
    # Initialize a single title object that we can update efficiently
    title = ax.set_title("Animation of Detected Marker Positions")

    # --- Animation Update Function ---
    def update(frame_index):
        #print(f"frame {frame_index}") # You can uncomment this for debugging
        current_positions = positions_history[frame_index]
        
        visible_coords = []
        
        # Loop through all possible markers and update their text artists
        for marker_id, pos in enumerate(current_positions):
            text_artist = texts[marker_id]
            if pos is not None:
                visible_coords.append(pos)
                text_artist.set_position((pos[0], pos[1] - 20))
                text_artist.set_visible(True)
            else:
                text_artist.set_visible(False)
        
        if visible_coords:
            scatter.set_offsets(np.array(visible_coords))
        else:
            scatter.set_offsets(np.empty((0, 2)))

        title.set_text(f"Animation of Detected Marker Positions (Frame: {frame_index + 1}/{len(positions_history)})")

        # Return all artists that could have been modified
        return [scatter, title] + texts

    # --- Create and Display the Animation ---
    ani = animation.FuncAnimation(
        fig=fig, 
        func=update, 
        frames=len(positions_history), 
        interval=50, 
        blit=False 
    )

    # --- Display the Animation ---
    # FIX: Using an explicit display call with HTML can be more reliable
    # in some Jupyter environments than relying on the default display.
    plt.close(fig) # Prevent the initial static plot from showing up
    display(HTML(ani.to_jshtml()))
